In [1]:
import sympy as sym
import numpy as np 
from gpkit import ureg as u

from Stability import *
from Performance import *

# TODO: place in each state also the aerodynamic terms that change with flight segment (i.e. cd0)
state = [
   {"U": None,"CD0": 0.049777, "rho": 1.225, "T": None, "Mass": None}, # takeoff
#   {"U": , "rho": 0.653, "T": , "Mass": }, # climb (density average of takeoff and cruise)
   {"U": 221 ,"CD0": 0.02674,"rho": 0.311, "T": None, "Mass": None}, # cruise (speed: M.75, alt: 40000ft)
#   {"U": , "rho": , "T": , "Mass": }, # descent
#   {"U": , "rho": , "T": , "Mass": }, # climb
   {"U": 137.2 ,"CD0": 0.0276,"rho": 0.525, "T": None, "Mass": None}, # 2nd cruise (speed: M.4, 26000ft)
#   {"U": None, "CD0": None, "rho": None, "T": None, "Mass": None}, # loiter (min Drag speed)
#   {"U": , "rho": , "T": , "Mass": }, # descent
   {"U": None, "CD0": 0.097632, "rho": 1.225 , "T": None, "Mass": None} # landing
]


# TODO: THIS IS ONLY FOR CRUISE! do for landing + takeoff (prob just change x_cg and lift curve slopes)
l_h = 6.8 * u.m
h_h = 4 * u.m
aircraft_x_cg = 6.32 * u.m
x_np = 0.0 * u.m
wing_b = 13 * u.m
wing_AR = 9
wing_qc = 6.40 * u.m
fuselage_length = 13.5 * u.m
fuselage_width = 2.0 * u.m
wing_c = 1.83 * u.m
wing_taper = 0.32
wing_S = 22 * u.m**2
wing_sweep = 9.1 * u.deg # for cm0_w sweep is of the 1/4 c position!
wing_cl_alpha = 6.4968 * (1/u.rad)
eta_h = 1
S_h = 6.503 * u.m**2
compressibility_correction = 1.31
CL_alpha_h = 3.72 * 1/u.rad
x_ac_h = 13 * u.m
wing_alpha0 = -1.8 * u.deg
wing_iw = 2 * u.deg
tail_alpha0 = 0 * u.deg
wing_twist = -3 * u.deg
x_aerocenter_tail = 13.5 * u.m
elevator_deflection_conver_parameter = 0.2

# Are these the same, are they different? Who knows ....this is the question 
CM0_w = -0.0445 
wing_cm0_af = -0.0445

# TODO: doing cruise right now, should do all the flight segments  
oswald_wing = 0.8
oswald_tailplane = 0.82 
tail_AR = 3
aircraft_Cd0 = 0.02674
cl_delta_e = 0.8 / (35 * u.deg).to(u.rad)  # 25 or 35 


"""
  aircraft_x_cg,
    wing_qc,
    x_aerocenter_tail,
    l_h,
    h_h,
    wing_AR,
    wing_taper,
    wing_c,
    wing_S,
    wing_b,
    wing_sweep,
    wing_cl_alpha,
    fuselage_length,
    fuselage_width,
    eta_h,
    S_h,
    CL_alpha_h,
    compressibility_correction,
    wing_alpha0,
    wing_iw,
    tail_alpha0,
    cl_delta_e,
    wing_cm0_af, # CM0_af is the incompressible airfoil zero lift pitching moment
    wing_twist,
    oswald_wing,
    oswald_tailplane,
    tail_AR,
    aircraft_Cd0
"""

stab = Stability(
    aircraft_x_cg,
    wing_qc,
    x_aerocenter_tail,
    l_h,
    h_h,
    wing_AR,
    wing_taper,
    wing_c,
    wing_S,
    wing_b,
    wing_sweep,
    wing_cl_alpha,
    fuselage_length,
    fuselage_width,
    eta_h,
    S_h,
    CL_alpha_h,
    compressibility_correction,
    wing_alpha0,
    wing_iw,
    tail_alpha0,
    cl_delta_e,
    wing_cm0_af, # CM0_af is the incompressible airfoil zero lift pitching moment
    wing_twist,
    oswald_wing,
    oswald_tailplane,
    tail_AR,
    aircraft_Cd0,
    CM0_w,
    elevator_deflection_conver_parameter
)

stab.stability_analysis()
# after getting the mass from mission performance stab.trim_analysis(state)


# THESE ARE VALUES ARE FOR CRUISE WHICH IS FINE FOR POINT PERFORMANCE (what about mission?)
aircraft_W0 = 5064 * u.kg
aircarft_Wlanding = 0.95 * aircraft_W0 # worst case scenario
aircraft_Cd0_landing = 0.097632
aircraft_Cd0_takeoff = 0.049777
aircraft_Cl_ground = 2.087
aircraft_Cl_landing = 2.44
aircraft_Cl_max_clean = 1.26
Cl_climb = 1.26 # same as cruise currently, only used for BFL 
wing_oswald = 0.8 # for cruise config 
aircraft_W0_S = 2227 * (u.n / u.m**2)
aircraft_W_S_landing = aircraft_W0_S * (aircraft_Wlanding/aicraft_W0) # is this legal?
aircraft_ground_AoA = 0.0 # initial angle to rotate --> 0
aircraft_takeoff_AoA = (13 * u.deg).to(u.rad) # angle we climb at with flaps 
h_obs_takeoff = (35 * u.ft).to(u.meter)
h_obs_landing = (35 * u.ft).to(u.meter)
engine_BPR = 2.8
n_engines = 2
aircraft_LoverD_takeoff = 18.5;
ac_reverse_thrust = False
ac_wheel_braking = True
T_max = 16239
static_T_takeoff = 14823
ac_stall_speed_takeoff = (2*aircraft_W0)/(wing_S*1.225*aircraft_Cl_ground)
ac_stall_speed_landing = (2*aircraft_Wlanding)/(wing_S*1.225*aircraft_Cl_landing)
dragOEI_evaluated_at_Vcl = 0.5 * state[0]["rho"] * wing_S * aircraft_Cd0_takeoff * (1.2 * ac_stall_speed_takeoff) ** 2 
aircraft_T0_W0 = 0.28
takeoff_density = 1.225
#takeoff_density = 1.225 (u.kg / u.m**3)

perf = Performance(
ac_stall_speed_takeoff,
ac_stall_speed_landing,
aircraft_Cd0_landing,
aircraft_Cd0_takeoff,
aircraft_Cl_ground,
aircraft_CL_landing,
aicraft_Cl_max_clean,
Cl_climb,
wing_AR,
wing_oswald,
aircraft_W0_S,
aircraft_W_S_landing,
aircraft_ground_AoA,
aircraft_takeoff_AoA,
h_obs_takeoff,
h_obs_landing,
engine_BPR,
n_engines,
aircraft_W0,
aircarft_Wlanding,
aircraft_LoverD_takeoff,
ac_reverse_thrust,
ac_wheel_braking,
T_max,
static_T_takeoff,
dragOEI_evaluated_at_Vcl,
aircraft_T0_W0,
takeoff_density,
wing_S
)

# Doing ground perf and updating states with takeoff and landing Velocity
state = perf.ground_performance(state)

# Getting thrust for each segment
state = stab.calc_thrust(state)

# Getting the mission profile 
state = perf.mission_profile(state)

# showing state with all values
print(state)

# Getting the trim analysis results 
stab.trim_analysis(state)

perf.point_performance()

ModuleNotFoundError: No module named 'sympy'